In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt

In [2]:
df = pd.read_csv('./data/review_data_2t.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141254 entries, 0 to 141253
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   review  141254 non-null  object
 1   시설분류    141254 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [5]:
df.head(2)

,review,시설분류
0,?,문화
1,친절해요,문화


In [6]:
df.columns = ['review','label']

In [7]:
df.head(2)

,review,label
0,?,문화
1,친절해요,문화


In [8]:
# 정규 표현식을 사용하여 한글만 추출
def extract_korean(text):

    # 한글과 공백만 추출
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]','',text)
    # 공백 2개 이상인경우 1개로 변환
    text = re.sub(r'\s+',' ',text)

    return text

In [27]:
# 형태소 분석 및 특정 품사 추출 함수
def extract_pos(text) :
    okt = Okt()
    pos_data = okt.pos(text)
    adj = ' '.join([word for word, tag in pos_data if tag in ['Adjective']])
    verb = ' '.join([word for word, tag in pos_data if tag in ['Verb']])
    noun = ' '.join([word for word, tag in pos_data if tag in ['Noun']])
    exclamation = ' '.join([word for word, tag in pos_data if tag in ['Exclamation']])
    
    # 각 품사에 대해 값이 없는 경우 NaN 할당
    adj = adj if adj else np.nan
    verb = verb if verb else np.nan
    noun = noun if noun else np.nan
    exclamation = exclamation if exclamation else np.nan
    
    return adj, verb, noun, exclamation                        
                            

In [28]:
# 한글만 추출하여 새로운 컬럼에 저장
df['review2'] = df['review'].map(extract_korean)

In [38]:
df_test = df.head(5).copy()
display(df_test)
df_test[['adj','verb','noun','exclamation']] = df_test['review2'].apply(extract_pos).apply(pd.Series)
df_test

,review,label,review2
0,?,문화,
1,친절해요,문화,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요
3,와우~~,문화,와우
4,굿,문화,굿


,review,label,review2,adj,verb,noun,exclamation
0,?,문화,,NaN,NaN,NaN,NaN
1,친절해요,문화,친절해요,친절해요,NaN,NaN,NaN
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,NaN,NaN,NaN
3,와우~~,문화,와우,NaN,NaN,와우,NaN
4,굿,문화,굿,NaN,NaN,굿,NaN


In [ ]:
from tqdm import tqdm
tqdm.pandas()
# 형용사,동사,명사, 감탄사 추출
df[['adj','verb','noun','exclamation']] = df['review2'].progress_apply(extract_pos).progress_apply(pd.Series)

 76%|███████████████████████████████████████████████████████▊                 | 107898/141254 [04:51<01:25, 392.41it/s]

In [ ]:
df.to_csv('./data/review_data_2t_1.csv',index=False)